Imports

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd
import requests
import re

Opening The Url

In [2]:
url = urlopen("http://www.northsouth.edu/")
nsu = soup(url.read())
#Examples of how to find by tag
#links = nsu.find_all('a')
#scripts = nsu.find_all('script')

Going through all the links

In [3]:
links = []

for link in nsu.find_all('a'):
  if 'href' in link.attrs:
    links.append((link.attrs['href']))

Creating dataframe for the links

In [4]:
links_df = pd.DataFrame(links)
links_df.head(2)

#Nameing the columns
links_df.columns = ['links']
links_df.head(2)


,links
0,contact-us.html
1,archive.html


Modifying the dataframe

In [5]:
# Removing empty links from the dataframe
links_df = links_df[links_df["links"].str.contains("#") == False]
links_df.head(2)

,links
0,contact-us.html
1,archive.html


In [6]:
# Removing external links from the data frame
internal_links = links_df[links_df["links"].str.contains("http") == False]
internal_links.head(2)

,links
0,contact-us.html
1,archive.html


In [7]:
# Adding Https infront of all the addresses
internal_links['links'] = 'http://www.northsouth.edu/' + internal_links['links'].astype(str)
internal_links.head(2)

/var/folders/_w/kwxvx6js1gvfr3r6sz47fpmc0000gn/T/ipykernel_10823/3951678606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internal_links['links'] = 'http://www.northsouth.edu/' + internal_links['links'].astype(str)


,links
0,http://www.northsouth.edu/contact-us.html
1,http://www.northsouth.edu/archive.html


In [8]:
# Creating new dataframe for external links
external_links = links_df[links_df["links"].str.contains("http") == True]
external_links.head(2)

,links
5,http://library.northsouth.edu
7,http://institutions.northsouth.edu/oea/interna...


Getting all the text data from the homepage

In [9]:
for data in nsu(['style', 'script', 'a', 'footer', 'nav']):
  # Remove tags
  data.decompose()

data = nsu.get_text()
data = data.replace("\n", " ")
#data = nsu.findAll('p')
data = re.sub(' +',' ',data)

data

' Home | North South University NSU Ranked #1 In Bangladesh with the following international university ranking agencies 1. Times Higher Education (THE): 2023 (Public/Private Universities) 2. QS World University Ranking: 2022, 2021, 2020 (Private) 3. QS Asia University Ranking: 2023, 2022, 2021, 2020, 2019 (Private) 4. Webometrics University Ranking: 2022 (Private) Undergraduate Study Graduate Study Research Notice 05/04/2023 04/04/2023 27/03/2023 14/03/2023 13/03/2023 '

Creating a new list for all data of different website pages

In [10]:
website_data = pd.DataFrame({'address': ['http://www.northsouth.edu/'],
                             'title': ['Home | North South University'],
                             'data': [data]})
#website_data[data].replace("\n"," ")
website_data

#website_data.to_csv('scrapper_data.csv', mode='w', index=False, header=False)
website_data.to_csv('scrapper_data.csv', mode='w', index=False)

Interlink and External Link seperated into their own list

In [11]:
internal_link_list = []
external_link_list = []

for item in links:
  if "http" in item:
    external_link_list.append(item)
  elif "#" in item:
    continue
  elif "pdf" in item:
    continue
  else:
    internal_link_list.append(item)


Adding full address to all internal links

In [12]:
internal_link_list_full = []
for item in internal_link_list:
  temp = "http://www.northsouth.edu/" + item
  internal_link_list_full.append(temp)

print(len(internal_link_list_full))


95


Getting All Links

In [13]:
# def get_links_from_list(url_list, temp_url_list):
#   for url in url_list:
#     headers = {"User-Agent":"Mozilla/5.0"}
#     # start session and get the search page
#     session = requests.Session()
#     response = session.get(url)

#     # parse the search page lxml
#     data = soup(response.content, 'lxml')
#     #print("Accessing URL -------------------------- " + tag)
    
#     for link in data.find_all('a'):
#       if 'href' in link.attrs:
#           temp = link.attrs['href']
#           #print("Url Found - " + temp)
#           if "http" in temp:
#             continue
#           elif "#" in temp:
#             continue
#           elif "pdf" in temp:
#               continue
#           elif "//" in temp:
#             continue
#           elif "gmail" in temp:
#             continue
#           elif "mailto" in temp:
#             continue
#           elif "jpg" in temp:
#             continue
#           elif "jpeg" in temp:
#             continue
#           elif "png" in temp:
#             continue
#           elif "doc" in temp:
#             continue
#           elif "docx" in temp:
#             continue
#           elif "xls" in temp:
#             continue
#           elif "pptx" in temp:
#             continue
#           elif "~" in temp:
#             continue
#           elif "http://www.northsouth.edu/" + temp in temp_url_list:
#             continue
#           else:
#             #print("Appending Link ----- http://www.northsouth.edu/" +temp)
#             temp_url_list.append("http://www.northsouth.edu/" + temp)
#             with open(csv_name,'a') as fd:
#               fd.write("http://www.northsouth.edu/" + temp + "\n")  

#   print("def get links from list temp list size")
#   print(len(temp_url_list))

In [14]:
# Getting links from the links in internal links list
# temp_url_list_1 = []
# temp_url_list_2 = []

# get_links_from_list(internal_link_list_full, temp_url_list_1)
# get_links_from_list(temp_url_list_1, temp_url_list_2)

In [15]:
def get_links_from_url(url_list, temp_url_list):
  for url in url_list:
    headers = {"User-Agent":"Mozilla/5.0"}
    # start session and get the search page
    session = requests.Session()
    response = session.get(url)

    # parse the search page lxml
    data = soup(response.content, 'lxml')
    
    for link in data.find_all('a'):
      if 'href' in link.attrs:
          temp = link.attrs['href']
          
          if "http" in temp:
            continue
          elif "#" in temp:
            continue
          elif "pdf" in temp:
              continue
          elif "//" in temp:
            continue
          elif "gmail" in temp:
            continue
          elif "mailto" in temp:
            continue
          elif "jpg" in temp:
            continue
          elif "jpeg" in temp:
            continue
          elif "png" in temp:
            continue
          elif "doc" in temp:
            continue
          elif "docx" in temp:
            continue
          elif "xls" in temp:
            continue
          elif "pptx" in temp:
            continue
          elif "~" in temp:
            continue
          
          else:
            temp_url_list.add("http://www.northsouth.edu/" + temp)


  print("def get links from list temp list size")
  print(len(temp_url_list))

In [18]:
# Getting links from the links in internal links list
temp_url_set_1 = set()
temp_url_set_2 = set()

get_links_from_url(internal_link_list_full, temp_url_set_1)
get_links_from_url(temp_url_set_1, temp_url_set_2)

def get links from list temp list size
733
def get links from list temp list size
2007


In [19]:
print(len(temp_url_set_1))
print(len(temp_url_set_2))

733
2007


In [22]:
session = requests.Session()

for tag in temp_url_set_2:

    # start session and get the search page
    response = session.get(tag)

    # parse the search page lxml
    data = soup(response.content, 'lxml')
    print("Accessing URL -------------------------- " + tag)
    
    if data.find('title') is None:
        continue
    elif data.find('title').string is None:
        continue
    else:
        title = data.find('title').string
    
        for item in data(['style', 'script', 'a', 'footer', 'nav', 'title']):
            # Remove tags
            item.decompose()
        
        #print(title)
    
        temp_data = data.get_text()
        temp_data = temp_data.replace("\n", " ")
        temp_data = re.sub(' +',' ',temp_data)
        print(temp_data)
    
        temp_df = pd.DataFrame({'address': [tag],
                                'title': [title],
                                'data': [temp_data]})
        
        #pd.concat([website_data, temp_df], ignore_index=True)
        #website_data = website_data.append({'title': title}, ignore_index=True)
        #website_data = website_data.append({'data': temp_data}, ignore_index=True)
        #website_data = website_data.append({'address': tag, 'title': title, 'data': temp_data}, ignore_index=True)
        temp_df.to_csv('scrapper_data.csv', mode='a', index=False, header=False)    

Accessing URL -------------------------- http://www.northsouth.edu/faculty-members/shss/pss/?page=3
 Center of Excellence in Higher Education The First Private University in Bangladesh Department of Political Science & Sociology - Faculty Members Lecturer (On Leave) Master of Culture Health and Medicine, Australian National University, Australia MSS in Anthropology, Jahangirnagar University, Bangladesh 	 Lecturer M.A., Simon Fraser University, Canada B.Sc., Quest University Canada, Canada 	 LecturerM.A., Swansea University, UK 	 Professor (Part Time Faculty) Ph.D., University of Delhi, India M.Sc., London University, UK 	 Professor (Part Time Faculty)Ph.D., University of Pune, India M. Phil, University of Bergen, Norway 	 Professor (Part Time Faculty) Ph.D., University of Aix- Marseille lll, France M.A., Dalhousie University, Canada 	 Professor (Part Time Faculty) PhD., Durham University, UK M.Sc., University of Birth, UK 	 Professor (Part Time Faculty) M.Sc., London School of Economic

KeyboardInterrupt: 